# 数据预处理
:label:`sec_pandas`

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
我们将在后面的章节中介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [39]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [40]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
#print(data)，输出直接用data的话会使用jupyter自己的HTML格式，更好看一些
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。


In [41]:
#回顾语法，[[:, 0:2]]，:代表所有行,0:2是左开右闭。
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
#用指定方法填充Na/NaN
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


/var/folders/l5/z95gbpr13d3bjrhwpn7xfl700000gn/T/ipykernel_39316/2509160536.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [42]:
#把离散值（分类值）转变为dummy值
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。


In [43]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。


In [44]:
#test1
data_file = os.path.join('..', 'data', 'pd_test1.csv')
with open(data_file, 'w') as t:
    #注意，里面填充数据的时候不要加空格，不然识别不了NA
    t.write('animals,nums,dummy\n')
    t.write('dog,3,NA\n')
    t.write('cat,3,NA\n')
    t.write('worm,5,NA\n')
    t.write('ququ,NA,NA\n')

data_t1 = pd.read_csv(data_file)
print(data_t1)
#开始处理缺省值
nan_num = data_t1.isna().sum(axis=0) #找出每一列NA的数量（理解axis=0的含义
print(nan_num)
na_maxid = nan_num.idxmax() #找出NA最多的那一列
data_t1 = data_t1.drop([na_maxid], axis=1) #理解这个axis=1
print(data_t1)


  animals  nums  dummy
0     dog   3.0    NaN
1     cat   3.0    NaN
2    worm   5.0    NaN
3    ququ   NaN    NaN
animals    0
nums       1
dummy      4
dtype: int64
  animals  nums
0     dog   3.0
1     cat   3.0
2    worm   5.0
3    ququ   NaN


In [48]:
#test2
#T2 = torch.tensor(data_t1.values) 注意，tensor不支持NA昂
ip2 = data_t1.iloc[:, 1] #注意，csv格式不能直接用索引，要加位置索引 iloc,但是tensor可以直接访问
#print(ip2)
#print(data_t1.iloc[:, 1])
data_t1.iloc[:, 1] = data_t1.iloc[:, 1].fillna(data_t1.iloc[:, 1].mean())
print(data_t1)
#T2 = torch.tensor(data_t1.values) #注意，tensor同时也不支持分类值（只支持数字和布尔）
ip2 = pd.get_dummies(data_t1.iloc[: ,0:-1])
print(ip2)
Ip, Op = torch.tensor(ip2.values), torch.tensor(data_t1.iloc[:, 1])
Ip, Op



  animals      nums
0     dog  3.000000
1     cat  3.000000
2    worm  5.000000
3    ququ  3.666667
   animals_cat  animals_dog  animals_ququ  animals_worm
0            0            1             0             0
1            1            0             0             0
2            0            0             0             1
3            0            0             1             0


(tensor([[0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0]], dtype=torch.uint8),
 tensor([3.0000, 3.0000, 5.0000, 3.6667], dtype=torch.float64))

[Discussions](https://discuss.d2l.ai/t/1750)
